In [1]:
import openpyxl, csv
import random
from openpyxl import Workbook, load_workbook

Convert xlsx & csv data to txt data 

In [2]:
root_path = "..\\data"

chatbot_raw_data = root_path + "\\ChatbotData .csv"
wellness_raw_data = root_path + "\\wellness_dialog_dataset.xlsx"

wellness_qna_data = root_path + "\\wellness_dialog_for_autoregressive.txt"
wellness_category_data = root_path + "\\wellness_data_for_text_classification.txt"
chatbot_wellness_merged = root_path + "\\chatbot_wellness_dialog_for_autoregressive.txt"

Save wellness_auto_regressive.txt data

In [3]:
#Make sure to encode txt files with UTF-8 
wb_well = load_workbook(filename=wellness_raw_data)
ws_well = wb_well[wb_well.sheetnames[0]]

f_well = open(wellness_qna_data, "w")
f_cat = open(wellness_category_data, "w")
f_chat_well = open(chatbot_wellness_merged, "w")

type_buffer = None
q_buffer = []
a_buffer = []

for row in ws_well.iter_rows():
    if row[0].value == '구분':
        continue
    else:
        f_cat.writelines(row[0].value + "\t" + row[1].value + "\n") # save category-question pairs
    if type_buffer == row[0].value or type_buffer is None:
        type_buffer = row[0].value
        q_buffer.append(row[1].value)
        if row[2].value is not None:
            a_buffer.append(row[2].value)
    else:
        for i in range(len(q_buffer)):
            for j in range(len(a_buffer)):
                line_to_write = q_buffer[i] + "\t" + a_buffer[j] + "\n"
                f_well.writelines(line_to_write)
                f_chat_well.writelines(line_to_write) # save question-answer pairs
        # initialize buffers and add new values
        type_buffer = row[0].value
        q_buffer = []
        a_buffer = []
        q_buffer.append(row[1].value)
        if row[2].value is not None:
            a_buffer.append(row[2].value)

# further read 'Chatbot .csv' file to add more question-answer pairs
with open(chatbot_raw_data, 'r', encoding='UTF8') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        if row[0] == 'Q':
            continue
        line_to_write = row[0] + "\t" + row[1] + "\n"
        f_chat_well.writelines(line_to_write) # save question-answer pairs

f_well.close()
f_cat.close()
f_chat_well.close()